In [ ]:
!nvidia-smi

Tue Jan  4 12:39:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --quiet transformers==4.5.0
!pip install --quiet pytorch-lightning==1.2.7

In [ ]:
!pip install --upgrade simplet5

  Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
  Using cached pytorch_lightning-1.4.5-py3-none-any.whl (919 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.0
    Uninstalling transformers-4.5.0:
      Successfully uninstalled transformers-4.5.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.2.7
    Uninstalling pytorch-lightning-1.2.7:
      Successfully uninstalled pytorch-lightning-1.2.7


In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
pl.seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Fetch data into daataframe
data_frame = pd.read_excel(r'/content/drive/MyDrive/Data/data_new_T5.xlsx')
data_frame = data_frame.rename({'input_text': 'source_text', 'target_text': 'target_text'}, axis=1)
data_frame.drop('Unnamed: 0', axis =1, inplace=True)

In [ ]:
data_frame

,source_text,target_text
0,5.30pm Bangladesh have won the toss and will b...,"Bangladesh 122 for 8 (Naim 23, Ellis 2-16, Chr..."
1,"Toss Bangladesh win toss, will bat first - the...",Australia 105 for 7 (Christian 39) beat Bangla...
2,"The ball too. Shakib is off for a run, but is ...","Bangladesh 127 for 9 (Mahmudullah 52, Ellis 3-..."
3,5.30pm Australia win the toss and bat first to...,Bangladesh 123 for 5 (Afif 37*) beat Australia...
4,But stopped by short third. Nobody there so Ag...,"Bangladesh 131 for 7 (Shakib 36, Naim 30, Hazl..."
...,...,...
458,"Clarke to Mahela Jayawardene, no run pushed of...","Australia 232 for 3 (Ponting 66*, Symonds 63*)..."
459,"Tait to K O'Brien, (no ball) oh dear. Tait to ...","Australia 92 for 1 (Gilchrist 34, Hussey 30*) ..."
460,I like to call that a Test cut. England win th...,"Australia 248 for 3 (Ponting 86, Clarke 55*) b..."
461,"Good ball all the same. Watson to Vaas, (no ba...","Australia 102 for 0 (Hayden 58*, Gilchrist 31*..."


In [ ]:
train_df = data_frame.iloc[:32, :]
eval_df = data_frame.iloc[391:(391+49), :]
test_df = data_frame.iloc[(391+49):,:]

In [ ]:
from simplet5 import SimpleT5

Global seed set to 42


In [ ]:
# instantiate
model = SimpleT5()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
history = model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 4,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# load trained T5 model
model.load_model("t5","/content/outputs/simplet5-epoch-4-train-loss-2.8235", use_gpu=False)

# predict
# lst_summ = []
# for i in range(281, 286):
#   lst_summ.append()
# print(model.predict(data_frame.iloc[260,0]))

In [ ]:
from matplotlib import pyplot

pyplot.plot(history[1]['train_loss'], label='train')
pyplot.plot(history[1]['eval_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Use the model for prediction
model_out_put = []

for i in range(test_df.shape[0]):
  model_out_put.append(model.predict(
                test_df.iloc[i,0]
        
    ))

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
reference_summary = test_df.iloc[:,1]

In [ ]:
reference_summary = reference_summary.to_list()

In [ ]:
import pandas as pd
lst_of_df = []
for i in range(len(reference_summary)):
  model_out = model_out_put[i]
  reference = reference_summary[i]
  lst_of_df.append(pd.DataFrame(rouge.get_scores(model_out[0], reference)[0]))

In [ ]:
df1 = lst_of_df[0]
df2 = lst_of_df[1]

In [ ]:
for i in range(2,len(lst_of_df)):
  df2 = df2.add(df1, fill_value = 0)
  df1 = lst_of_df[i]

In [ ]:
df2/test_df.shape[0]

,rouge-1,rouge-2,rouge-l
r,0.154390,0.024997,0.121077
p,0.313649,0.058599,0.252252
f,0.202061,0.034138,0.159793


In [ ]:
pd.DataFrame(rouge.get_scores(model_out, reference)[0])

With masked

In [ ]:
# Fetch data into daataframe
data_frame = pd.read_excel(r'/content/drive/MyDrive/Data/data_new_T5_masked.xlsx')
data_frame = data_frame.rename({'input_text': 'source_text', 'target_text': 'target_text'}, axis=1)
data_frame.drop('Unnamed: 0', axis =1, inplace=True)

In [ ]:
train_df = data_frame.iloc[:32, :]
eval_df = data_frame.iloc[391:(391+49), :]
test_df = data_frame.iloc[(391+49):,:]

In [ ]:
# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
history = model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 4,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# load trained T5 model
model.load_model("t5","/content/outputs/simplet5-epoch-4-train-loss-2.8235", use_gpu=False)

# predict
# lst_summ = []
# for i in range(281, 286):
#   lst_summ.append()
# print(model.predict(data_frame.iloc[260,0]))

In [ ]:
from matplotlib import pyplot

pyplot.plot(history[1]['train_loss'], label='train')
pyplot.plot(history[1]['eval_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Use the model for prediction
model_out_put = []

for i in range(test_df.shape[0]):
  model_out_put.append(model.predict(
                test_df.iloc[i,0]
        
    ))

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
reference_summary = test_df.iloc[:,1]

In [ ]:
reference_summary = reference_summary.to_list()

In [ ]:
import pandas as pd
lst_of_df = []
for i in range(len(reference_summary)):
  model_out = model_out_put[i]
  reference = reference_summary[i]
  lst_of_df.append(pd.DataFrame(rouge.get_scores(model_out[0], reference)[0]))

In [ ]:
df1 = lst_of_df[0]
df2 = lst_of_df[1]

In [ ]:
for i in range(2,len(lst_of_df)):
  df2 = df2.add(df1, fill_value = 0)
  df1 = lst_of_df[i]

In [ ]:
df2/test_df.shape[0]

,rouge-1,rouge-2,rouge-l
r,0.154390,0.024997,0.121077
p,0.313649,0.058599,0.252252
f,0.202061,0.034138,0.159793
